In [1]:
import numpy as np
import pandas as pd
from neighbor_knn import NeighborKNN

/Users/jehunyoo/AI/kakao/MelonPlaylistContinuation/main/neighbor_knn.py:7: UserWarning: Unsupported module 'tqdm' is used.
  warn("Unsupported module 'tqdm' is used.")


In [4]:
### 1. load data
train = pd.read_json("res/train.json")
val = pd.read_json("res/val.json")
pred_songs = pd.read_json("submission/neighbor2_a7b0/neighbor2_a7b0_first.json", orient='records')
pred_tags = pd.read_json("submission/neighbor-tag_beta1_a7b0/neighbortag_beta1_a7b0.json", orient='records')
# test = pd.read_json("res/test.json")

In [19]:
index = []
for i in val.index:
    if val.songs[i] != [] and val.tags[i] == []:
        index.append(i)
print(len(index))
new_val = val.iloc[index, :]
display(new_val)
new_val.shape[0]

9661


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
6,[],127575,,"[227044, 424672, 515592, 22408, 57893, 153714,...",2,2011-10-28 11:21:51.000
...,...,...,...,...,...,...
23003,[],127349,,"[474446, 111476, 241237, 70500, 577313, 145058...",14,2018-02-02 17:43:37.000
23006,[],92067,,"[486938, 154124, 70969, 596414, 243733, 557848...",9,2013-02-16 01:07:41.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


9661

In [6]:
### 2. modeling
### 2.1 hyperparameters: k, rho, alpha, beta
### 2.2 parameters: sim_songs, sim_tags, sim_normalize
k = 100
rho = 0.4
alpha = 0.5
beta = 0.5
sim_songs = "idf"
sim_tags = "cos"
sim_normalize = False

In [21]:
### 3. range setting - KNN.predict()
### 3.1 range(start, end); if end == None, then range(start, end of val)
### 3.2 auto_save: boolean; False(default)
### 3.3 return type of KNN.predict() : pandas.DataFrame
pred = NeighborKNN(k=k, rho=rho, alpha=alpha, beta=beta, \
           sim_songs=sim_songs, sim_tags=sim_tags, sim_normalize=sim_normalize, \
           train=train, val=new_val, pred_songs=pred_songs, pred_tags=pred_tags, \
           verbose=True, version_check=True).predict(start=0, end=None, auto_save=False)
display(pred)

KNN version: NeighborKNN-1.0


  0%|          | 1/9661 [00:03<10:05:07,  3.76s/it]


KeyError: 1

In [ ]:
### 4. save data
version = NeighborKNN.__version__
version = version[version.find('-') + 1: version.find('.')]
path = "submission/neighbor-knn_k100rho4_idfcosFalse"
fname = f"neighbor-knn{version}_k{k}rho{int(rho * 10)}_{sim_songs}{sim_tags}{sim_normalize}"
pred.to_json(f'{path}/{fname}.json', orient='records')

Only Tags

In [ ]:
import numpy as np
import pandas as pd
from neighbor_knn_tag import NeighborKNN_tag

In [3]:
### 1. load data
train = pd.read_json("res/train.json")
val = pd.read_json("res/val.json")
pred_songs = pd.read_json("submission/neighbor2_a7b0/neighbor2_a7b0_first.json", orient='records')
pred_tags = pd.read_json("submission/neighbor-tag_beta1_a7b1/neighbortag_beta1_a7b1.json", orient='records')
# test = pd.read_json("res/test.json")

In [22]:
index = []
for i in val.index:
    if val.songs[i] != [] and val.tags[i] == []:
        index.append(i)
new_val = val.iloc[index, :].copy()
new_val

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
6,[],127575,,"[227044, 424672, 515592, 22408, 57893, 153714,...",2,2011-10-28 11:21:51.000
...,...,...,...,...,...,...
23003,[],127349,,"[474446, 111476, 241237, 70500, 577313, 145058...",14,2018-02-02 17:43:37.000
23006,[],92067,,"[486938, 154124, 70969, 596414, 243733, 557848...",9,2013-02-16 01:07:41.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [7]:
### 2. modeling
### 2.1 hyperparameters: k, rho, alpha, beta
### 2.2 parameters: sim_songs, sim_tags, sim_normalize
k = 100
rho = 0.4
sim_songs = "idf"
sim_tags = "cos"
sim_normalize = False

In [17]:
### 3. range setting - KNN.predict()
### 3.1 range(start, end); if end == None, then range(start, end of val)
### 3.2 auto_save: boolean; False(default)
### 3.3 return type of KNN.predict() : pandas.DataFrame
pred = NeighborKNN(k=k, rho=rho, \
           sim_songs=sim_songs, sim_normalize=sim_normalize, \
           train=train, val=val, pred_songs=pred_songs, \
           verbose=True, version_check=True).predict(start=0, end=None, auto_save=False)
display(pred)

KNN version: NeighborKNN-1.0


KeyboardInterrupt: 

In [16]:
import numpy as np
import pandas as pd
import os
from collections import Counter
from warnings import warn

warn("Unsupported module 'tqdm' is used.")
from tqdm.notebook import tqdm


class NeighborKNN:

    __version__ = "NeighborKNN-1.0"
    
    def __init__(self, k, rho=0.4, \
                 sim_songs="idf", sim_normalize=False, \
                 train=None, val=None, \
                 pred_songs=None, \
                 verbose=True, version_check=True):
        '''
        k : int
        rho : float; 0.4(default) only for idf
        alpha, beta : float; 0.5(default)
        sim_songs, sim_tags : "cos"(default), "idf", "jaccard"
        sim_normalize : boolean; when sim == "cos" or "idf"
        verbose : boolean
        '''
        self.train_id = train["id"].copy()
        self.train_songs = train["songs"].copy()
        self.train_tags = train["tags"].copy()
        del train

        self.val_id = val["id"].copy()
        self.val_songs = val["songs"].copy()
        self.val_tags = val["tags"].copy()
        del val
        
        self.pred_songs = pred_songs["tags"].copy()

        self.freq_songs = None
        self.freq_tags = None
        
        self.k = k
        self.rho = rho
        self.alpha = alpha
        self.beta = beta

        self.sim_songs = sim_songs
        self.sim_normalize = sim_normalize

        self.verbose = verbose
        self.__version__ = NeighborKNN.__version__

        if version_check:
            print(f"KNN version: {NeighborKNN.__version__}")

        TOTAL_SONGS = 707989      # total number of songs

        if self.sim_songs == "idf":

            self.freq_songs = np.zeros(TOTAL_SONGS, dtype=np.int64)
            _playlist = self.train_songs
            for _songs in _playlist:
                self.freq_songs[_songs] += 1


    def predict(self, start=0, end=None, auto_save=False, auto_save_step=500, auto_save_fname='auto_save'):
        '''
        start, end : range(start, end). if end = None, range(start, end of val)
        auto_save : boolean; False(default)
        auto_save_step : int; 500(default)
        auto_save_fname : string (without extension); 'auto_save'(default)
        @returns : pandas.DataFrame; columns=['id', 'songs', 'tags']
        '''

        # TODO: Remove unsupported module 'tqdm'.
        if end:
            _range = tqdm(range(start, end)) if self.verbose else range(start, end)
        elif end == None:
            _range = tqdm(range(start, self.val_id.index.stop)) if self.verbose else range(start, self.val_id.index.stop)

        pred = []
        all_songs = [set(songs) for songs in self.train_songs] # list of set
        all_tags =  [set(tags) for tags in self.train_tags]    # list of set

        for uth in _range:

            k = self.k
            
            # predict tags using songs
            if self.val_songs[uth] != [] and self.val_tags[uth] == []:
                playlist_songs = set(self.pred_songs[uth])
                simSongs = np.array([self._sim(playlist_songs, vplaylist, self.sim_songs, opt="songs") for vplaylist in all_songs])
                tags = []
                
                while len(tags) < 10:
                    top = simSongs.argsort()[-k:]
                    _tags = []
                    
                    for vth in top:
                        _tags += self.train_tags[vth]

                    counts = Counter(_tags).most_common(30)
                    tags = [tag for tag, _ in counts]

                    k += 100
                
                pred_tags = tags[:10]

                pred.append({
                "id" : int(self.val_id[uth]),
                "songs" : self.pred_songs[uth],
                "tags" : pred_tags
                })

            if (auto_save == True) and ((uth + 1) % auto_save_step == 0):
                self._auto_save(pred, auto_save_fname)
        
        return pd.DataFrame(pred)
    

    def _sim(self, u, v, sim, opt):
        '''
        u : set (playlist in train data)
        v : set (playlist in test data)
        sim : string; "cos", "idf", "jaccard" (kind of similarity)
        opt : string; "songs", "tags"
        '''

        if sim == "cos":
            if self.sim_normalize:
                try:
                    len(u & v) / ((len(u) ** 0.5) * (len(v) ** 0.5))
                except:
                    return 0
            else:
                return len(u & v)
        
        elif sim == "idf":
            if opt == "songs":
                freq = self.freq_songs
            elif opt == "tags":
                freq = self.freq_tags
            freq = freq[list(u & v)]
            freq = 1 / (((freq - 1) ** self.rho) + 1) # numpy!
            if self.sim_normalize:
                try:
                    return freq.sum() / ((len(u) ** 0.5) * (len(v) ** 0.5))
                except:
                    return 0
            else:
                return freq.sum()
        
        elif sim == "jaccard":
            return len(u & v) / len(u | v)
    
    def _auto_save(self, pred, auto_save_fname):
        '''
        pred : list of dictionaries
        auto_save_fname : string
        '''
        
        if not os.path.isdir("./_temp"):
            os.mkdir('./_temp')
        pd.DataFrame(pred).to_json(f'_temp/{auto_save_fname}.json', orient='records')

/Users/jehunyoo/anaconda3/envs/cosmos/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Unsupported module 'tqdm' is used.
  import sys
